In [6]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import gensim.downloader
import torch


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from dataset import EssayDataset, essay_collate_fn

In [3]:
BATCH_SIZE = 128
np.random.seed(42)

In [ ]:
#Loading Toeknized train Data from Palash's file
data = pd.read_csv("train_tokenized.csv") # Palash's file
data.drop(columns=["Unnamed: 0"], inplace=True)
data.head()

In [5]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
glove_dict = glove_vectors.key_to_index
len(glove_dict)

400000

In [10]:
#Vector Array
gensim_emb_weights = glove_vectors.vectors
gensim_emb_weights.shape

In [21]:
data['list_lower_word_token_nltk'] = data['word_token_nltk'].apply(eval)
data['list_lower_word_token_nltk'] = data['list_lower_word_token_nltk'].apply(lambda l: [e.lower() for e in l if e not in ('', ' ')])
data['num_valid_token'] = data['list_lower_word_token_nltk'].apply(lambda x: len([c for c in x if c in glove_dict]))
data['num_all_token'] = data['list_lower_word_token_nltk'].apply(len)
data['unrecgonized_tokens'] = data['list_lower_word_token_nltk'].apply(lambda x: [c for c in x if c not in glove_dict])
data["word_token_nltk_idx"] = data['list_lower_word_token_nltk'].apply(
    lambda x: [glove_vectors.key_to_index[c]  if c in glove_dict else 400000 for c in x])

In [22]:
data["word_token_nltk_idx"]

0       [41, 269, 12, 543, 54, 2161, 25, 2741, 22, 163...
1       [61, 7, 747, 14, 7, 511, 81, 33, 4, 886, 20, 8...
2       [8607, 1, 2965, 83, 6479, 511, 0, 164, 527, 3,...
3       [0, 254, 79, 6, 214, 14, 61, 81, 399, 4961, 2,...
4       [357, 743, 3, 20524, 86, 1433, 6, 68, 69, 86, ...
                              ...                        
3906    [41, 733, 622, 28319, 6, 905, 10, 631, 95, 91,...
3907    [500, 1485, 1, 543, 88, 36, 33, 4, 4117, 17, 6...
3908    [28, 7, 747, 14, 7, 1019, 10, 81, 4, 88, 392, ...
3909    [109, 69, 10027, 17, 4324, 28833, 9, 8643, 1, ...
3910    [88, 81, 269, 12, 1899, 14, 0, 444, 873, 10, 6...
Name: word_token_nltk_idx, Length: 3911, dtype: object

In [23]:
task_cols = ["cohesion","syntax","vocabulary", 
    "phraseology", "grammar", "conventions"]
data.loc[1, task_cols].to_list()

[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]

In [24]:
df_train, df_valid, df_test = np.split(
    data[["text_id", "full_text", "word_token_nltk_idx"] + task_cols].sample(
        frac=1, random_state=42), 
    [int(.8*len(data)), int(.9*len(data))])

In [25]:
df_train.head()

,text_id,full_text,word_token_nltk_idx,cohesion,syntax,vocabulary,phraseology,grammar,conventions
1552,772D27D400BB,It god to have a possitive attitude when you d...,"[20, 1533, 4, 33, 7, 400000, 4191, 61, 81, 88,...",3.0,2.5,2.5,2.0,2.0,2.0
2114,9E8F3C6405CA,Why do people ask more then one person for adv...,"[738, 88, 69, 1712, 56, 127, 48, 899, 10, 3240...",3.0,2.0,3.0,3.5,3.0,3.0
1965,948771F795EB,"We accomplish more when we are active, and are...","[53, 9749, 56, 61, 53, 32, 1546, 1, 5, 32, 690...",4.0,4.0,3.0,4.0,4.0,4.0
3856,FE14D7378CFB,Do you agree or disagree about imagination bei...,"[88, 81, 2137, 46, 10027, 59, 9201, 134, 56, 4...",3.0,3.0,3.5,3.0,3.5,3.5
1610,7AAE019F70D6,I disagree with the principal saying that all ...,"[41, 10027, 17, 0, 2965, 345, 12, 64, 1813, 18...",3.5,3.5,3.5,3.5,3.0,3.5


In [26]:
ds_train = EssayDataset(df_train, "word_token_nltk_idx", task_cols)
dl_train = torch.utils.data.DataLoader(
    ds_train, batch_size=BATCH_SIZE, 
    shuffle=True, collate_fn=essay_collate_fn)
ds_valid = EssayDataset(df_valid, "word_token_nltk_idx", task_cols)
dl_valid = torch.utils.data.DataLoader(
    ds_valid, batch_size=BATCH_SIZE, 
    shuffle=True, collate_fn=essay_collate_fn)
ds_test = EssayDataset(df_test, "word_token_nltk_idx", task_cols)
dl_test = torch.utils.data.DataLoader(
    ds_test, batch_size=BATCH_SIZE, 
    shuffle=True, collate_fn=essay_collate_fn)

In [39]:
df_test.shape

(392, 9)

In [27]:
from gru import GRUGrader
from trainer import train_grader

In [28]:
gru_grader = GRUGrader(gensim_emb_weights=gensim_emb_weights)


In [29]:
best_default_model = train_grader(dl_train, dl_valid, 
    model=gru_grader,
    model_name="default",
    loss_func=torch.nn.MSELoss(),
    opt=torch.optim.Adam(gru_grader.parameters(), lr=0.001),
    epochs=30)

Epoch 0000, train loss: 0.917489, valid loss: 0.799257, best valid loss: inf
Epoch 0001, train loss: 0.438210, valid loss: 0.407726, best valid loss: inf
Epoch 0002, train loss: 0.438662, valid loss: 0.401707, best valid loss: inf
Epoch 0003, train loss: 0.422672, valid loss: 0.462174, best valid loss: 0.462174
Epoch 0004, train loss: 0.409525, valid loss: 0.405165, best valid loss: 0.462174
Epoch 0005, train loss: 0.394810, valid loss: 0.347148, best valid loss: 0.462174
Epoch 0006, train loss: 0.367131, valid loss: 0.453248, best valid loss: 0.453248
Epoch 0007, train loss: 0.349584, valid loss: 0.328271, best valid loss: 0.453248
Epoch 0008, train loss: 0.331219, valid loss: 0.308093, best valid loss: 0.453248
Epoch 0009, train loss: 0.324908, valid loss: 0.353262, best valid loss: 0.353262
Epoch 0010, train loss: 0.318262, valid loss: 0.314473, best valid loss: 0.353262
Epoch 0011, train loss: 0.308925, valid loss: 0.323371, best valid loss: 0.323371
Epoch 0012, train loss: 0.30623

In [92]:
best_default_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_pred = torch.concat([
    best_default_model([xb.to(device), sb.to(device)]) for ([xb, sb]), _ in dl_test]
    ).cpu().detach().numpy()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:954: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
  self.num_layers, self.dropout, self.training, self.bidirectional)


**Performance Evaluation**

In [93]:
## Converting Prediction array into Pandas Dataframe

pred_cols = ["cohesion_pred","syntax_pred","vocabulary_pred", 
    "phraseology_pred", "grammar_pred", "conventions_pred"]

pred_df = pd.DataFrame(data= test_pred, columns = pred_cols)
pred_df.head(5)

,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.651461,3.454639,3.686070,3.594072,3.442849,3.682776
1,3.388020,3.194679,3.441223,3.347355,3.158853,3.313161
2,3.533994,3.355825,3.565684,3.471856,3.393566,3.590303
3,3.921016,3.705667,3.914559,3.844837,3.682505,3.977976
4,3.367429,3.191929,3.433972,3.355413,3.229849,3.421072


In [95]:
##Joining Prediction Results to Actual test Data
final_test = (df_test.reset_index()).join(pred_df)
final_test.shape

(392, 16)

In [106]:
## Importing Evaluation Metrics
from eval_metric import mcrmse, quad_kappa,round_to_multiple

# Quadratic Weighted Kappa metric is used for categorical responses. In order to use it for our use case, following transofrmation were performed on the predicted outcome of measures:
# 1. Rounding the predicted values to closest multiple of 0.5.
# 2. converting to string.
# 3. Using cohen_kappa_score function from sklearn.metrics


In [107]:
actual_cols = ["cohesion","syntax","vocabulary", "phraseology", "grammar", "conventions"]
pred_cols = ["cohesion_pred","syntax_pred","vocabulary_pred", "phraseology_pred", "grammar_pred", "conventions_pred"]
mcrmse = mcrmse(final_test, actual_cols, pred_cols)
print(mcrmse)

0.5477241043296427


In [108]:
quad_kappa = quad_kappa(final_test, actual_cols, pred_cols)
print(quad_kappa)

[0.036703624027800696, -0.005929000222337555, 0.0369465483074084, 0.0002808199943835632, 0.029363556408411262, 0.029099170288423792]
